# How To Use Vertex Gemini Vision Pro Generative AI To Inspect Image Details

This notebook outlines how to interact with Vertex AI's Gemini Vision Pro GenAI model to inspect images and generate detailed information about its content. Visual Question Answering (VQA) lets you provide an image to the model and ask a question about the image's contents. In response to your question you get one or more natural language answers.

## Prepare the python development environment

First, let's identify any project specific variables to customize this notebook to your GCP environment. Change YOUR_PROJECT_ID with your own GCP project ID.

In [ ]:
PROJECT_ID = 'rkiles-demo-host-vpc'
LOCATION = 'us-central1'

Next, let's specify the name of the image file you want to inspect, such as "OJ.png" or "shoe.png"

In [ ]:
image_filename = 'OJ.png'

Install any needed python modules from our requirements.txt file. Most Vertex Workbench environments include all the packages we'll be using, but if you are using an external Jupyter Notebook or require any additional packages for your own needs, you can simply add them to the included requirements.txt file an run the folloiwng commands.

In [ ]:
#pip install -r requirements.txt

Now we will import all required modules. For our purpose, we will be utilizing the following:

- google.auth - Provides authentication access to the Google API's, such as imagegeneration:predict
- base64 - Imagen API requests return generated or edited images as base64-encoded strings. This module will help us decode this data to an image file
- requests - This module will allow us to interact directly with Imagen over the REST API. 
- json - Python module used to interact with JSON data. Imagen returns results in json format.

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import base64
import json

## Authenticate to the Vertex AI API

Our Vertex Workbench instance is configured to use a specified service account that has IAM access to the Gemini Vison Pro API. The following two secitons will allow us to generate the access token we will pass as an authorization bearer token later in the header.

To use Gemini Vision Pro on Vertex AI you must provide a text description of what you want to inspect, generate or edit. These descriptions are called prompts, and these prompts are the primary way you communicate with Generative AI. Here, we are specifiying what we want the model to identify using a prompt. Play around with this content and see what kind of details you can extract from an image. More information can be found here https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview

In this example, we will ask Gemini to inspect a picture of an orange juice carton and provide it's results in a json format.

In [ ]:
vqa_prompt = 'Briefly describe each product you see in this picture and provide your response in JSON format including the brand, description and size. If you can not determine the size, mark it as NA'

In [ ]:
#def generate_text(project_id: str, location: str) -> str:
#    # Initialize Vertex AI
#    vertexai.init(project=project_id, location=location)
#    # Load the model
#    multimodal_model = GenerativeModel("gemini-pro-vision")
#    # Query the model
#    response = multimodal_model.generate_content(
#        [
#            # Add an example image
#            Part.from_uri(
#                "gs://generativeai-downloads/images/scones.jpg", mime_type="image/jpeg"
#            ),
#            # Add an example query
#            "what is shown in this image?",
#        ]
#    )
#    #print(response)
#    return response.text

In [ ]:
with open(image_filename, "rb") as f:
    encoded_base_image = base64.b64encode(f.read())
B64_BASE_IMAGE = encoded_base_image.decode('utf-8')

In [ ]:
def generate_text(project_id: str, location: str, b64_image: str, prompt: str) -> str:
    # Initialize Vertex AI
    vertexai.init(project=project_id, location=location)
    # Load the model
    multimodal_model = GenerativeModel("gemini-pro-vision")
    # Query the model
    response = multimodal_model.generate_content(
        [
            # Add an example image
            Part.from_data(
                data=base64.b64decode(b64_image), mime_type="image/png"
            ),
            #"what is shown in this image?",
            vqa_prompt,
        ]
    )
    #print(response)
    return response.text

## Prepare the HTTP POST request to the REST API

Define the header fields, including the access token we created in the last step

In [ ]:
qa_response = json.loads(generate_text(PROJECT_ID, LOCATION, B64_BASE_IMAGE, vqa_prompt))
print(qa_response)

That's it! Congratulations on defining your first visual Q&A with Imagen!